In [22]:
import random

NUM = 100 # 每次生成100道题
TYPE_DICT = ['mul', 'div']

MUL1_MIN = 10
MUL1_MAX = 100
MUL2_MIN = 2
MUL2_MAX = 9

DIV1_MIN = 20
DIV1_MAX = 150
DIV2_MIN = 1
DIV2_MAX = 9

In [23]:
type_rnd = random.randint(0, len(TYPE_DICT))
mul1 = random.randint(MUL1_MIN, MUL1_MAX)
mul2 = random.randint(MUL2_MIN, MUL2_MAX)

mul_res = mul1 * mul2

div1 = random.randint(DIV1_MIN, DIV1_MAX)
div2 = random.randint(DIV2_MIN, DIV2_MAX)

div_res = div1//div2

div_rem = div1 % div2

print(TYPE_DICT[type_rnd])
print(f"mul1 = {mul1}, mul2 = {mul2} mul_res = {mul_res}")
print(f"div1 = {div1}, div2 = {div2} div_res = {div_res} div_rem ={div_rem}")


mul
mul1 = 88, mul2 = 2 mul_res = 176
div1 = 35, div2 = 9 div_res = 3 div_rem =8


In [ ]:
# def problem_gen(type_rnd = 'mul'):

In [94]:
def mul_problem_gen (index:int, mul1:int, mul2:int, debug: bool = False):
    mul_res = mul1 * mul2
    format_str = f"{index:<4} | {mul1:<4} x {mul2:>4} = {' ' * 10} |{' ' *5} 答案：{mul_res}"
    if debug: print(format_str)
    return format_str
    

In [95]:
mul_problem_gen(1,18,3, True)
mul_problem_gen(122,55,5, True)

1    | 18   x    3 =            |      答案：54
122  | 55   x    5 =            |      答案：275


'122  | 55   x    5 =            |      答案：275'

In [96]:
def div_problem_gen (index:int, div1:int, div2:int,  debug: bool = False):

    div_res = div1//div2

    div_rem = div1 % div2
    
    ans_str = f"{div_res}" if div_rem == 0 else f"{div_res}...{div_rem}" 
    
    format_str = f"{index:<4} | {div1:<4} \u00F7 {div2:>4} = {' ' * 10} |{' ' *5} 答案：{ans_str}"
    if debug: print(format_str)
    return format_str

In [97]:
div_problem_gen(1,18,3, True)
div_problem_gen(12,55,15, True)

1    | 18   ÷    3 =            |      答案：6
12   | 55   ÷   15 =            |      答案：3...10


'12   | 55   ÷   15 =            |      答案：3...10'

In [98]:
def problem_gen (type_: str, index: int):
    problem_str = None
    if type_ == 'mul':
        mul1 = random.randint(MUL1_MIN, MUL1_MAX)
        mul2 = random.randint(MUL2_MIN, MUL2_MAX)
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        div1 = random.randint(DIV1_MIN, DIV1_MAX)
        div2 = random.randint(DIV2_MIN, DIV2_MAX)
        problem_str = div_problem_gen(index, div1, div2)
    return problem_str
        

In [110]:
def print_problem_single(round_:int):
    for i in range (round_):
#         row = 0
        type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        
        print_str = problem_gen(type_, i+1)
        
        print(f"{print_str:<50}", end='\n')


In [107]:
def print_problem_double(round_:int):
    for i in range (round_):
        row = 0
        
        type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        
        print_str = problem_gen(type_, i+1)
        
        print(f"{print_str:<50}", end='|  ')
        if i % 2 == 1: #每行两个题目
            row += 1

            print("\n")  # 每两个题目换行
            print("\n")  # 每两个题目加一空行
            
        i += 1

In [109]:
print_problem_double(NUM)

1    | 65   x    2 =            |      答案：130     |  2    | 48   x    9 =            |      答案：432     |  



3    | 37   x    5 =            |      答案：185     |  4    | 74   ÷    7 =            |      答案：10...4  |  



5    | 22   x    8 =            |      答案：176     |  6    | 66   ÷    9 =            |      答案：7...3   |  



7    | 29   ÷    1 =            |      答案：29      |  8    | 31   x    3 =            |      答案：93      |  



9    | 19   x    5 =            |      答案：95      |  10   | 27   ÷    1 =            |      答案：27      |  



11   | 78   ÷    1 =            |      答案：78      |  12   | 146  ÷    5 =            |      答案：29...1  |  



13   | 85   x    3 =            |      答案：255     |  14   | 98   x    4 =            |      答案：392     |  



15   | 45   x    2 =            |      答案：90      |  16   | 60   x    8 =            |      答案：480     |  



17   | 62   x    6 =            |      答案：372     |  18   | 25   ÷    9 =            |      答案：2...7   |  



19   | 32 

In [118]:
def print_problem_daily(days:int, nums:int):
    for d in range(days):
        print("日期_______， 正确数目_______, 错误数目_______, 得分_______")
        print_problem_single(nums)
        print("\n"*2)
    


In [122]:
print_problem_daily(3, 30)

日期_______， 正确数目_______, 错误数目_______, 得分_______
1    | 126  ÷    1 =            |      答案：126     
2    | 105  ÷    5 =            |      答案：21      
3    | 87   x    2 =            |      答案：174     
4    | 25   ÷    9 =            |      答案：2...7   
5    | 23   x    3 =            |      答案：69      
6    | 54   ÷    2 =            |      答案：27      
7    | 98   ÷    6 =            |      答案：16...2  
8    | 77   x    9 =            |      答案：693     
9    | 96   x    6 =            |      答案：576     
10   | 50   x    6 =            |      答案：300     
11   | 62   ÷    4 =            |      答案：15...2  
12   | 34   ÷    8 =            |      答案：4...2   
13   | 68   ÷    5 =            |      答案：13...3  
14   | 133  ÷    6 =            |      答案：22...1  
15   | 55   ÷    8 =            |      答案：6...7   
16   | 68   x    7 =            |      答案：476     
17   | 59   ÷    6 =            |      答案：9...5   
18   | 40   x    3 =            |      答案：120     
19   | 53   ÷    9 =            |  